In [ ]:
import numpy as np
import pickle
import os

seed = 2023

In [ ]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from nlpsig_networks.scripts.seqsignet_attention_functions import (
    seqsignet_attention_hyperparameter_search,
)

In [ ]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [ ]:
%run load_sbert-embeddings.py

In [ ]:
df_rumours.head()

# SeqSigNet with Attention Network

In [ ]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
num_features = len(features)
add_features_in_path = True

In [ ]:
num_epochs = 100
dimensions = [15]  # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (
    df_rumours[df_rumours["set"] == "train"].index,
    df_rumours[df_rumours["set"] == "dev"].index,
    df_rumours[df_rumours["set"] == "test"].index,
)

In [ ]:
shift = 3
window_size = 5
n = 11

In [ ]:
size = shift * n + (window_size - shift)
print(size)

## UMAP

In [ ]:
(
    seqsignet_attention_umap,
    best_seqsignet_attention_umap,
    _,
    __,
) = seqsignet_attention_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_indices=split_indices,
    k_fold=False,
    features=features,
    standardise_method=standardise_method,
    add_features_in_path=add_features_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_attention_umap_focal_{gamma}_{size}.csv",
    verbose=False,
)

In [ ]:
seqsignet_attention_umap.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "learning_rate",
    ]
).mean()

In [ ]:
best_seqsignet_attention_umap

In [ ]:
best_seqsignet_attention_umap[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_seqsignet_attention_umap["f1"].mean()

In [ ]:
best_seqsignet_attention_umap["precision"].mean()

In [ ]:
best_seqsignet_attention_umap["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_umap["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_umap["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_umap["recall_scores"]).mean(axis=0)

## Random Projections

In [ ]:
(
    seqsignet_attention_grp,
    best_seqsignet_attention_grp,
    _,
    __,
) = seqsignet_attention_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_indices=split_indices,
    k_fold=False,
    features=features,
    standardise_method=standardise_method,
    add_features_in_path=add_features_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/seqsignet_attention_grp_focal_{gamma}_{size}.csv",
    verbose=False,
)

In [ ]:
seqsignet_attention_grp.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_seqsignet_attention_grp

In [ ]:
best_seqsignet_attention_grp.columns

In [ ]:
best_seqsignet_attention_grp[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_seqsignet_attention_grp["f1"].mean()

In [ ]:
best_seqsignet_attention_grp["precision"].mean()

In [ ]:
best_seqsignet_attention_grp["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_grp["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_grp["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_grp["recall_scores"]).mean(axis=0)